# Clasificador de imágenes

## Bibliotecas necesarias

In [7]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img
from keras.preprocessing import image
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPool2D
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import load_model

In [19]:
import os
import shutil
import random

os.makedirs("data/train/cat", exist_ok=True)
os.makedirs("data/train/dog", exist_ok=True)
os.makedirs("data/test/cat", exist_ok=True)
os.makedirs("data/test/dog", exist_ok=True)

all_images = os.listdir("train")

random.shuffle(all_images)

for img in all_images[:10000]:  # 5000 por clase para train
    label = img.split(".")[0]
    shutil.copy(f"train/{img}", f"data/train/{label}/{img}")

for img in all_images[10000:12000]:  # 1000 por clase para test
    label = img.split(".")[0]
    shutil.copy(f"train/{img}", f"data/test/{label}/{img}")


FileNotFoundError: [Errno 2] No such file or directory: 'train'

In [20]:
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def mostrar_imagenes(clase, path="data/train", n=9):
    files = os.listdir(f"{path}/{clase}")[:n]
    plt.figure(figsize=(10, 10))
    for i, file in enumerate(files):
        img = load_img(f"{path}/{clase}/{file}", target_size=(200, 200))
        plt.subplot(3, 3, i+1)
        plt.imshow(img)
        plt.axis("off")
    plt.suptitle(f"Imágenes de {clase}")
    plt.show()

mostrar_imagenes("cat")
mostrar_imagenes("dog")


<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

In [21]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    "data/train",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_data = test_datagen.flow_from_directory(
    "data/test",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 0 images belonging to 2 classes.
Found 0 images belonging to 2 classes.


In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense

model = Sequential()
model.add(Conv2D(input_shape = (224,224,3), filters = 64, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 64,kernel_size = (3,3),padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size = (2,2),strides = (2,2)))
model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size = (2,2),strides = (2,2)))
model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size = (2,2),strides = (2,2)))
model.add(Flatten())
model.add(Dense(units = 4096,activation = "relu"))
model.add(Dense(units = 4096,activation = "relu"))
model.add(Dense(units = 2, activation = "softmax"))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


/home/vscode/.local/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-05-20 20:21:38.972457: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


: 

In [1]:
history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=5  # Puedes usar más si tienes tiempo
)


NameError: name 'model' is not defined

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("best_model.keras", monitor='val_loss', save_best_only=True)
earlystop = EarlyStopping(monitor='val_loss', patience=3)

model.fit(
    train_data,
    validation_data=test_data,
    epochs=10,
    callbacks=[checkpoint, earlystop]
)


In [ ]:
model.save("final_model.keras")
